In [1]:
import pdf2image
import os
import glob
import cv2
import numpy as np
import layoutparser as lp

### Open pdf

In [2]:
path_pdf = "/resources/datasets/image_recognition/sentencias/FALLO_TOMO CAF 023440_2022_CS001.pdf"
doc = pdf2image.convert_from_path(path_pdf)

In [3]:
len(doc)

22

### Create png by page

In [4]:
OUTPUT_PATH = '/resources/datasets/image_recognition/sentencias/images/'

In [5]:
def create_image_from_image(pdf_path: str):
    image_path = pdf2image.convert_from_path(
        pdf_path,
        dpi=600,
        output_folder=OUTPUT_PATH,
        fmt='png',
        paths_only=True
    )[0]

    target_name = f'{os.path.basename(pdf_path)}.png'
    target_path = f'{OUTPUT_PATH}/{target_name}'
    os.rename(image_path, target_path)

In [6]:
create_image_from_image(path_pdf)

### Open images

In [7]:
images = [cv2.imread(file) for file in glob.glob("/resources/datasets/image_recognition/sentencias/images/*.png")]

### Analize image by model NewspaperNavigator

In [14]:
model = lp.Detectron2LayoutModel(
   "lp://NewspaperNavigator/faster_rcnn_R_50_FPN_3x/config",
    # "lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config",
   extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
   label_map={0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"})
    # label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})

config.yml?dl=1: 8.19kB [00:03, 2.26kB/s]
model_final.pth?dl=1: 330MB [00:44, 7.42MB/s]                               


In [15]:
imgs_array = [np.asarray(image) for image in images]

In [16]:
layourt = [model.detect(img) for img in imgs_array]

In [17]:
len(imgs_array), len(layourt)

(22, 22)

In [18]:
OUTPUT_PATH_ANALIZED = f'/resources/datasets/image_recognition/sentencias/images_detecter_newspapaer/'

In [19]:
for i in range(len(imgs_array)):
    lp.draw_box(imgs_array[i], layourt[i], box_width=5, box_alpha=0.2, id_font_size=80,
            show_element_type=True).save(f'{OUTPUT_PATH_ANALIZED}/{i}.png')

### Filter only images with ilustration

In [ ]:
for i in range(len(imgs_array)):
    for ly in layourt[i].get_info('type'):
        if ly in ['Figure']:
            print(i,ly)

In [ ]:
i = 12
lp.draw_box(imgs_array[i], layourt[i], box_width=5, box_alpha=0.2,id_font_size=80 ,show_element_type=True)

In [ ]:
i = 12
img = imgs_array[i]
detected = layourt[i]

In [ ]:
lp.draw_box(img, detected, box_width=5, box_alpha=0.2, 
            show_element_type=True)

In [ ]:
dic_predicted = {}
for block in [block for block in detected if block.type in ["Title","Text"]]:
    ## segmentation
    segmented = block.pad(left=15, right=15, top=5, 
                bottom=5).crop_image(img)
    ## extraction
    extracted = model_ocr.detect(segmented)
    ## save
    dic_predicted[str(block.id)+"-"+block.type] = extracted.replace('\n',' ').strip()
# check
parse_doc(dic_predicted)